In [1]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import pandas as pd
import networkx as nx
from nltk.corpus import stopwords

In [2]:
def read_article(news_data, row):
#     print(news_data[row])
    article = news_data[row].split(". ")
    sentences = []

    for sentence in article:
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

In [5]:
news = pd.read_csv('ProcessedData.tsv', delimiter = '\t', encoding = 'utf-8')
news_data = news['body']

for row in range(len(news_data)):
    sentences =  read_article(news_data, row)

    # Step 2 - Generate Similary Martix across sentences
    stop_words = stopwords.words('english')
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    try:
        sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
        scores = nx.pagerank(sentence_similarity_graph)
    except:
        print("error at row - ", row)
        continue

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    # print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    summarize_text = []
    
    count = len(ranked_sentence)
    limit = 0
    if(count < 10):
        limit = count 
    else:
        limit = 10
    for i in range(limit):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
    
    #  Step 5 - Offcourse, output the summarize texr
    print("Summarize Text \n", ". ".join(summarize_text))
    print('\n\n**END**\n\n')
    
print('Summarization complete!!')

The markets haven't been kind to Bitcoin in 2014. The currency reached a high of nearly $1,000 in January before falling to around $350 this month, a plunge of more than 60 percent. It would be easy to write Bitcoin off as a fad whose novelty has worn off. After all, dollars seem superior in almost every respect. They're accepted everywhere, they're convenient to use, and they have a stable value. Bitcoin is an inferior currency on all three counts. Bitcoin's detractors are making the same mistake as many Bitcoin fans  Yet it would be foolish to write Bitcoin off. The currency has had months-long slumps in the past, only to bounce back. More importantly, it's a mistake to think about Bitcoin as a new kind of currency. What makes Bitcoin potentially revolutionary is that it's the world's first completely open financial network. History suggests that open platforms like Bitcoin often become fertile soil for innovation. Think about the internet. It didn't seem like a very practical techno

Summarize Text 
 There are already some Bitcoin applications that allow customers to make transactions over the Bitcoin network without being exposed to fluctuations in the value of Bitcoin's currency. (Flickr/FastLizard4)  The first open financial network The Bitcoin network serves the same purpose as mainstream payment networks such as Visa or Western Union. Bitcoin is a payment network that happens to have its own currency, not the other way around. Bitcoin's detractors are making the same mistake as many Bitcoin fans  Yet it would be foolish to write Bitcoin off. Bitcoin is different. Bitcoin is such a good deal for merchants that they may find it worthwhile to offer their customers discounts for paying with Bitcoin instead of cash or credit cards One obvious application is international money transfers. But there's no organization to perform this role on a peer-to-peer network like Bitcoin. In contrast, Bitcoin transactions only take about 30 minutes to clear, and Bitcoin transact